In [66]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [67]:
#Read the dataset file
data = pd.read_csv('/content/Iris.csv', index_col = 0)

In [68]:
#Display the first five rows of the dataframe
print(data.head())

    SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
Id                                                                       
1             5.1           3.5            1.4           0.2  Iris-setosa
2             4.9           3.0            1.4           0.2  Iris-setosa
3             4.7           3.2            1.3           0.2  Iris-setosa
4             4.6           3.1            1.5           0.2  Iris-setosa
5             5.0           3.6            1.4           0.2  Iris-setosa


In [69]:
#Displaying the information of the dataframe
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1 to 150
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 7.0+ KB
None


In [70]:
print(data.shape)

(150, 5)


In [71]:
print(data.isna().sum())

SepalLengthCm    0
SepalWidthCm     0
PetalLengthCm    0
PetalWidthCm     0
Species          0
dtype: int64


In [72]:
print(data.duplicated(keep = False).value_counts())
print(data[data.duplicated(keep = False)])

False    145
True       5
dtype: int64
     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm         Species
Id                                                                           
10             4.9           3.1            1.5           0.1     Iris-setosa
35             4.9           3.1            1.5           0.1     Iris-setosa
38             4.9           3.1            1.5           0.1     Iris-setosa
102            5.8           2.7            5.1           1.9  Iris-virginica
143            5.8           2.7            5.1           1.9  Iris-virginica


In [73]:
data = data.drop_duplicates(keep = 'first').copy()
print(data.shape)

(147, 5)


In [74]:
data = data.reset_index(drop = True)

In [75]:
X = data.iloc[:, 0:4].values
y = data['Species'].values

In [76]:
print(X.shape)
print(y.shape)

(147, 4)
(147,)


In [77]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.fit_transform(X)

In [78]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [79]:
num_categories = 3
y_onehot = keras.utils.to_categorical(y_encoded, num_categories)

In [80]:
#convert X_scaled and y_onehot into train and valid data

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y_onehot,
                                                     test_size = 0.2,
                                                     random_state = 42)

In [81]:
#Define a model

model = Sequential()
model.add(Dense(units = 512, activation = 'relu', input_shape = (4,)))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = num_categories, activation = 'softmax'))

In [82]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               2560      
                                                                 
 dense_13 (Dense)            (None, 512)               262656    
                                                                 
 dense_14 (Dense)            (None, 512)               262656    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 3)                 1539      
                                                                 
Total params: 529411 (2.02 MB)
Trainable params: 529411 (2.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [83]:
#Compiling the model

model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history=model.fit(X_train, y_train,
          epochs = 100,
          verbose = 1,
          validation_data = (X_valid, y_valid))

Epoch 1/100
4/4 [==============================] - 2s 97ms/step - loss: 0.9764 - accuracy: 0.5470 - val_loss: 0.8096 - val_accuracy: 0.6667
Epoch 2/100
4/4 [==============================] - 0s 23ms/step - loss: 0.6755 - accuracy: 0.6581 - val_loss: 0.5307 - val_accuracy: 0.7333
Epoch 3/100
4/4 [==============================] - 0s 23ms/step - loss: 0.5321 - accuracy: 0.7009 - val_loss: 0.4528 - val_accuracy: 0.8000
Epoch 4/100
4/4 [==============================] - 0s 22ms/step - loss: 0.4825 - accuracy: 0.7607 - val_loss: 0.5121 - val_accuracy: 0.6667
Epoch 5/100
4/4 [==============================] - 0s 23ms/step - loss: 0.4690 - accuracy: 0.7778 - val_loss: 0.3815 - val_accuracy: 0.8667
Epoch 6/100
4/4 [==============================] - 0s 22ms/step - loss: 0.3934 - accuracy: 0.8205 - val_loss: 0.3518 - val_accuracy: 0.9000
Epoch 7/100
4/4 [==============================] - 0s 24ms/step - loss: 0.3933 - accuracy: 0.7436 - val_loss: 0.3534 - val_accuracy: 0.8667
Epoch 8/100
4/4 [===

In [ ]:
#Print actual and predicted value

y_pred = model.predict(X_valid)

actual = np.argmax(y_valid, axis = 1)
predicted = np.argmax(y_pred, axis = 1)

print(f"Actual:    {actual}")
print(f"Predicted: {predicted}")